In [1]:
import csv
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import torch.nn.functional as F

In [2]:
import torch
import torch.nn as nn

In [3]:
# if torch.cuda.is_available():
#     device = torch.device("cuda")
# else:
#     device = torch.device("cpu")

device = torch.device("cpu")

In [4]:
data = []
min_len = float('inf')

In [5]:
with open('../data/2023-10-04-10-f.csv', 'r',newline='', encoding='utf-8') as csvfile:
    spikereader = csv.reader(csvfile, delimiter=';')
    for row in spikereader:
        data.append(row)
        if len(row) < min_len:
            min_len = len(row)

In [6]:
data = [[float(item) for item in sublist[:min_len]] for sublist in data]

In [7]:
data_tensor = torch.tensor(data).to(device)

In [8]:
print(data_tensor)
print(data_tensor.shape)

tensor([[   0., 1608., 1595.,  ..., 1631., 1601., 1610.],
        [   0., 1619., 1624.,  ..., 1617., 1600., 1580.],
        [   0., 1571., 1590.,  ..., 1575., 1588., 1615.],
        ...,
        [   9., 1501., 1565.,  ..., 1602., 1571., 1594.],
        [   9., 1575., 1581.,  ..., 1570., 1594., 1588.],
        [   9., 1581., 1556.,  ..., 1596., 1575., 1577.]])
torch.Size([100, 8601])


In [9]:
class CustomTensorDataset(Dataset):
    def __init__(self, data_tensor):
        self.data_tensor = data_tensor

    def __len__(self):
        return len(self.data_tensor)

    def __getitem__(self, idx):
        sample = self.data_tensor[idx]
        label = torch.tensor(int(sample[0])).to(device)
        item = sample[1:]
        return item, label

In [10]:
custom_dataset = CustomTensorDataset(data_tensor)

In [11]:
train_size = 0.6  # Proportion of the dataset for training
train_dataset, test_dataset = train_test_split(custom_dataset, train_size=train_size, shuffle=True)

In [12]:
batch_size = 2
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [13]:
class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, dropout_prob=0.0):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.dropout1 = nn.Dropout(dropout_prob)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm1d(out_channels)
        self.dropout2 = nn.Dropout(dropout_prob)
        self.downsample = None

        if stride != 1 or in_channels != out_channels:
            self.downsample = nn.Sequential(
                nn.Conv1d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm1d(out_channels),
            )

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.dropout1(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.dropout2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

class ResNet1D(nn.Module):
    def __init__(self, block, layers, num_classes=10, dropout_prob=0.0):
        super(ResNet1D, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv1d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm1d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0],dropout_prob=dropout_prob)
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2, dropout_prob=dropout_prob)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2, dropout_prob=dropout_prob)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2, dropout_prob=dropout_prob)
        self.avgpool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, block, out_channels, blocks, stride=1, dropout_prob=0.0):
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, dropout_prob))
        self.in_channels = out_channels
        for _ in range(1, blocks):
            layers.append(block(out_channels, out_channels, dropout_prob=dropout_prob))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

# Define the ResNet model
model = ResNet1D(BasicBlock, [2, 2, 2, 2], num_classes=10)

In [23]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv1d(1, 16, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool1d(kernel_size=2, stride=2)
        self.fc = nn.Linear(16 * 4300, num_classes)  # Adjust the output size for 10 classes

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = self.fc(x)
        return x

In [24]:
import numpy as np
from sklearn import metrics

In [25]:
num_classes = 10
all_acu = []
all_auc = []
all_f1 = []
for i in range(5):
    model = SimpleCNN(num_classes=num_classes)
    model.to(device)

    criterion = nn.CrossEntropyLoss()  # Cross-entropy loss for multiclass classification
    optimizer = optim.AdamW(model.parameters(), lr=5e-5)

    scheduler = StepLR(optimizer, step_size=20, gamma=1e-1)    
    best_acu = 0
    best_auc = 0
    best_f1 = 0
    epochs = 70
    for epoch in range(epochs):
        train_loss = 0
        model.train()  # Set the model to training mode
        for batch_data, batch_labels in train_dataloader:
            optimizer.zero_grad()  # Zero the gradients
            batch_data = torch.unsqueeze(batch_data,1)
            outputs = model(batch_data)  # Forward pass
            loss = criterion(outputs, batch_labels)  # Calculate loss
            train_loss += loss
            loss.backward()  # Backpropagation
            optimizer.step()  # Update weights

        print(train_loss.item())
        scheduler.step()  # Adjust learning rate
        model.eval()  # Set the model to evaluation mode
        val_loss = 0.0
        correct = 0
        total = 0
        labels = 0
        preds = 0
        probs = 0
        with torch.no_grad():
            for batch_data, batch_labels in test_dataloader:
                batch_data = torch.unsqueeze(batch_data,1)
                outputs = model(batch_data)
                val_loss += criterion(outputs, batch_labels).item()
                _, predicted = outputs.max(1)
                np_labels = batch_labels.detach().cpu().numpy()
                np_predictions = predicted.detach().cpu().numpy()
                if total > 0:
                    labels = np.concatenate((labels,np_labels),axis=0)
                    preds = np.concatenate((preds,np_predictions),axis=0)
                    probs = np.concatenate((probs, torch.softmax(outputs, dim=1).detach().cpu().numpy()),axis=0)
                else:
                    labels = np_labels
                    preds = np_predictions
                    probs = torch.softmax(outputs, dim=1).detach().cpu().numpy()
                    # all_outputs = outputs.detach().cpu().numpy()
                total += batch_labels.size(0)
                correct += predicted.eq(batch_labels).sum().item()

        avg_val_loss = val_loss / len(test_dataloader)
        accuracy = 100.0 * correct / total
        acu = metrics.balanced_accuracy_score(labels, preds) * 100
        f1 = metrics.f1_score(labels, preds, average='weighted')
        if num_classes > 2:
            auc = metrics.roc_auc_score(labels, probs, multi_class='ovo')
        else:
            auc = metrics.roc_auc_score(labels, preds)
        if acu > best_acu:
            best_acu = acu
        if auc > best_auc:
            best_auc = auc
        if f1 > best_f1:
            best_f1 = f1

        # print(f"Epoch [{epoch+1}/{epochs}] - Validation Loss: {avg_val_loss:.4f} - Validation Accuracy: {accuracy:.2f}% - sklearn Accuracy: {acu:.2f}%")
        print(f"Epoch [{epoch+1}/{epochs}] - Balanced Accuracy: {acu:.2f}% - AUC: {auc:.4f} - F1: {f1:.4f}")
    all_acu.append(best_acu)
    all_auc.append(best_auc)
    all_f1.append(best_f1)
    print(f"Best Accuracy: {best_acu:.2f}% - Best AUC: {best_auc:.4f} - Best F1: {best_f1:.4f}")
print("\r\n")
print(f"Mean Accuracy: {np.mean(all_acu):.2f}% - Mean AUC: {np.mean(all_auc):.4f} - Mean F1: {np.mean(all_f1):.4f}")

ValueError: torch.Size([2, 68800])